In [9]:
import re
import io

text_file = 'C:\Users\VKorzun\Desktop\wp.txt'

# 1 - Триграммы

In [10]:
word_list = []
def text_to_wordlist(text):
    regexp = u'[^а-яА-Яё]'
    sent = re.sub(regexp, ' ', text)
    return sent.lower().split()

with io.open(text_file, "r", encoding='utf-16-le') as text_file:
    text = text_file.read()
    word_list = text_to_wordlist(text)    

In [22]:
def get_trigrams(word):
    trigramms = []
    for i in range(len(word)-2):
        trigramms.append(word[i:i+3])
    return trigramms

In [27]:
print word_list[0]

##лев##


In [26]:
for i in range(len(word_list)):
    word_list[i] = '##'+word_list[i]+'##'

In [28]:
trigramm_dict = dict()
for word in word_list:
    trigrams = get_trigrams(word)
    for trigram in trigrams:
        if trigram in trigramm_dict.keys():
            trigramm_dict[trigram] = trigramm_dict[trigram] + 1
        else:
            trigramm_dict[trigram] = 1

In [29]:
sorted_trigramms = sorted(trigramm_dict.items(), key=lambda (k, v): v, reverse=True)

for tr in sorted_trigramms[:10]:
    print tr[0], tr[1]

о## 13712
и## 11407
а## 10653
е## 10076
##с 9891
##п 9809
##в 9625
##н 9456
##о 7748
я## 7109


In [30]:
all_tr = 0
for tr in sorted_trigramms:
    all_tr += tr[1]

In [66]:
def is_bad(word, p):
    trigramms = get_trigrams(u'##'+word+u'##')
    for tr in trigramms:     
        print trigramm_dict.get(tr), tr, 
        if trigramm_dict.get(tr) == None or float(trigramm_dict.get(tr)) / all_tr <  p :            
            print float(trigramm_dict.get(tr)) / all_tr
            return True
    return False

In [69]:
print is_bad(u'ошипка', 0.00002)

7748 ##о 15 #ош 47 оши 2 шип 2.67613304128e-06
True


# 2 - Бор

In [1]:
class Node:
    def __init__(self):
        self.term = False
        self.children = {}    
        
class NodeCrawl:
    def __init__(self, node, string, error):
        self.node = node
        self.string = string
        self.error = error

In [2]:
def insert_word(bor, word):
    root = bor
    for i in range(len(word)):
        if root.children.get(word[i]) == None:
            for j in range (i, len(word)):
                new_root = Node()
                root.children[word[j]] = new_root
                root = new_root
            root.term=True            
            break
        else:
            root = root.children[word[i]]            

In [60]:
def find_word(results,best, part):
    new_results = []
    
    for result in results:
#         print result.string, part
        if result.error >= best.error:
            continue
        if result.node.term and result.error + len(part) < best.error:
            best = result         
            best.error += len(part)
        for key in result.node.children.keys():
            new_node = None
            if len(part) > 0 and key == part[0]:
                new_node = NodeCrawl(result.node.children[key], result.string + key, result.error)                
            else:
                new_node = NodeCrawl(result.node.children[key], result.string + key, result.error + 1)
            new_results.append(new_node)
        
    if len(new_results) == 0:
        return best.string
    if len(part) > 0:
        part = part[1:]
    return find_word(new_results, best, part)
        

In [29]:
bor = Node()


In [30]:
insert_word(bor, 'kot')
insert_word(bor, 'kut')
insert_word(bor, 'pot')

In [43]:
find_word([NodeCrawl(bor, '', 0)], NodeCrawl(bor, '', 3),'pok')

'pot'

In [11]:
root = Node()
for word in word_list:
    insert_word(root, word)

In [61]:
print find_word([NodeCrawl(root, '', 0)], NodeCrawl(root, '', 6), u'ошипка')

ошибки


In [63]:
print find_word([NodeCrawl(root, '', 0)], NodeCrawl(root, '', 5), u'шизнь')

жизни


In [66]:
print find_word([NodeCrawl(root, '', 0)], NodeCrawl(root, '', 3), u'леф')

лет


In [67]:
print find_word([NodeCrawl(root, '', 0)], NodeCrawl(root, '', 8), u'комантир')

командира
